For Lee's procedure for cleaning pdb files taken from the database see [here](https://wiki.ch.ic.ac.uk/wiki/index.php?title=Guide_to_Creating_ONIOM_input_files_for_biomolecules)

Here we will perform an oniom calculation on gfp!

The strategy for such calculations is set out in the [wiki](https://wiki.ch.ic.ac.uk/wiki/index.php?title=Guide_to_Creating_ONIOM_input_files_for_biomolecules). 

First we download the pdb file from the [PDB_database](http://www.rcsb.org/pdb/home/home.do), making sure to select a high quality structure - high resolution and low r-value(?).


Useful code from a 2010 development version of biopython [here](http://biopython.org/wiki/GSOC2010_Joao#Hydrogenation_Report), the bizkit library used to construct an amber calculation [here](http://biopython.org/wiki/GSOC2010_Joao#Hydrogenation_Report) (bizkit is very similar in format to the [CCPN](http://www.ccpn.ac.uk/software/documentations) python library which is used in the analysis of NMR data, Amber's acpype [python script](https://code.google.com/p/acpype/) for calling Amber's Antechamber program to generate topologies (for use with gromacs but can I use it too?), the pdb2pqr [python script](http://www.poissonboltzmann.org/pdb2pqr/) to calculates pKa's using Propka.

There is also a [web-framework](http://www.ncbi.nlm.nih.gov/pubmed/22437851) that does something like this.

And there is the [MDanalysis](http://mdanalysis.googlecode.com/git-history/develop/package/doc/html/documentation_pages/overview.html) python library which has many useful tools

There is also [OEChem](http://www.ics.uci.edu/~dock/manuals/oechem/pyprog/pyprog.html) which is can load mol2 files and is an alternative to molmod

And there [MMTK](http://dirac.cnrs-orleans.fr/MMTK/) which contains routines for pdb loadings and manipulation as well as methods governing direct application of Amber forcefields

There is another web-framework based on [PDBReport](http://swift.cmbi.ru.nl/gv/pdbreport/) that reports on the errors present in a given pdb file, this looks useful. As does [WHY-NOT](http://www.cmbi.ru.nl/WHY_NOT2/)

General resources: http://blanco.biomol.uci.edu/wwwresources.html

In [3]:
cd oniom_bio/

/home/clyde/Dropbox/Project Stuff/Notebooks/oniom_bio


In [4]:
import Bio
from ase import Atom
#from ase_extensions import Atoms

def ase_read_pdb(pdb_fn):
    """Reads a PDB into an ASE atoms object"""
    
    parser = Bio.PDB.PDBParser()
    protein=parser.get_structure(pdb_fn.replace('.pdb',''), pdb_fn)
    protein_atoms = [a for a in protein.get_atoms()]
    
    atom_names = [a.element for a in protein_atoms]
    atom_coords = [a.coord for a in protein_atoms]
    
    ase_pdb = Atoms([Atom(nm,crds) for nm,crds in zip(atom_names,atom_coords)])
    
    return ase_pdb

In [3]:
#from simtk.openmm.app.pdbfile import PDBFile

In [4]:
#def expected_atoms(res_name, chain):
#    res_bonds = chain.topology._standardBonds[res_name]
#    unique_atoms = list(set([b[0] for b in res_bonds] + [b[1] for b in res_bonds]))
#    backbone_atoms = [a for a in unique_atoms if 'H' not in a and 'XT' not in a and '-' not in a]
#    return sorted(backbone_atoms)

In [5]:
#gfp_raw_v2 = PDBFile('1GFL.pdb')
#chain = gfp_raw_v2.topology._chains[0]
#residues = chain._residues

#non_standard = []
#for r in residues:
#    atoms = sorted([a.name for a in r._atoms])
#    if expected_atoms(r.name,chain) != atoms:
#        non_standard.append(r)

In [6]:
#expected_atoms(residues[65].name,chain), sorted([r.name for r in residues[65]._atoms])

In [7]:
#[e for e in sorted([r.name for r in residues[72]._atoms]) if e not in expected_atoms(residues[72].name,chain)]

In [8]:
#[(r.index,r.name) for r in non_standard]

This is not working as there are additional atoms present in the xml file and a non standard residue might have the same atoms but with a different connectivity? I.e. residue 66 is not turning up with the above method. Perhaps we can fix this method but initially I will see if I can use the ambertools.

In [9]:
#from Bio.PDB import PDBParser
#from Bio.PDB import PDBIO

In [10]:
#pdb_p = PDBParser()
#gfp_raw = pdb_p.get_structure('1GFL','1GFL.pdb')

In [11]:
#chains=gfp_raw.get_chains()
#c1=chains.next()

In [12]:
#for model in gfp_raw:
#    for chain in model:
#        if chain.id != c1.id:
#            model.detach_child(chain.id)

In [13]:
#w = PDBIO()
#w.set_structure(gfp_raw)
#w.save('extracted_1GFL.pdb')

Or maybe just Bizkit directly?

In [14]:
#import Biskit
#gfp_raw=Biskit.PDBModel('1GFL.pdb')

In [15]:
#gfp_raw=B.PDBModel('1GFL.pdb')
#gfp_only = gfp_raw.takeChains([0])

In [16]:
#non_std_index = [64,65,66]
#non_std_atom_index = gfp_only.res2atomIndices(non_std_index)
#for i in non_std_atom_index:
#    gfp_only.atoms['residue_name'][i] = 'CHR'


In [17]:
#non_standard_atom_index

In [18]:
#non_std_index = [64,65,66]
#non_std_atom_index = gfp_only.res2atomIndices(non_std_index)
#for i in non_std_atom_index:
#    gfp_only.atoms['residue_name'][i] = 'CHR'

In [19]:
#%qtconsole

In [20]:
import cc_utils
cc_utils

<module 'cc_utils' from '/home/clyde/Dropbox/Project Stuff/Code/Python/cc_utils/cc_utils/__init__.py'>

In [5]:
import os
import Biskit as B
#from cc_notebook  import unwind
from cc_utils.chem_utils import mol2_parse, mol2_replace, set_frag_atom_types, get_bad, get_ht_ind, set_nonstd_name
from pybel import readfile
from molmod.io.xyz import XYZReader
from cc_utils.red_tools_utils import run_red

In [6]:
pdb_code = '1GFL'
orig_pdb = pdb_code + '.pdb'
mod_pdb = 'mod_' + orig_pdb
prot_mod_pdb = 'prot_' + mod_pdb
prot_mod_pqr = prot_mod_pdb.replace('.pdb', '.pqr')
master_pdb = pdb_code + '_master.pdb'
master_xyz = pdb_code + '_master.xyz'

#TODO algorithmically find the non_standard residues
#fixer = PDBFixer(pdbid='1YRI')
#fixer.findNonstandardResidues()
#nonstandard_residues = fixer.nonstandardResidues
        
non_standard_index = [64,65,66]

gfp_raw=B.PDBModel(orig_pdb)
gfp_only = gfp_raw.takeChains([0])
nonstd = gfp_only.takeResidues(non_standard_index)
#nonstd.writePdb('raw_nonstd.pdb')

##overview

Input: Initial PDB + locations of nonstandard residues 
    
Combine contiguous non standard residues to make a new single non-standard residue
Extract non-standard residue + cap with ACE/NME to generate non-standard peptide
Protonate capped non-standard residue
Protonate Initial PDB structure (except for non-standard part)
Uncap capped protonated non-standard residue and insert into protonated initial PDB structure for final structure

Compute partial charges associated with non-standard peptide
Compute GAFF parameters associated with non-standard peptide
Determine parameters associated with Terminal atoms of non-standard residue

Combine partial charges, GAFF atom types / parameters and Gaff-Amber interaction parameters.

Initial .dat file
mol2 with connectivity  -> generation of .frcmod file containing parameters not present in .dat file
combined .dat and .frcmod for complete parameter set

In [23]:
import os
import Biskit as B
from cc_notebook import unwind
from cc_utils.chem_utils import mol2_parse, mol2_replace, set_frag_atom_types, get_bad, get_ht_ind, set_nonstd_name
from pybel import readfile
from molmod.io.xyz import XYZReader
from cc_utils.red_tools_utils import run_red

pdb_code = '1GFL'
orig_pdb = pdb_code + '.pdb'
mod_pdb = 'mod_' + orig_pdb
prot_mod_pdb = 'prot_' + mod_pdb
prot_mod_pqr = prot_mod_pdb.replace('.pdb', '.pqr')
master_pdb = pdb_code + '_master.pdb'

#TODO algorithmically find the non_standard residues -> chimera can do this via:
#http://www.cgl.ucsf.edu/pipermail/chimera-dev/2009/000605.html
non_standard_index = [64,65,66]

gfp_raw=B.PDBModel(orig_pdb)
gfp_only = gfp_raw.takeChains([0])
nonstd = gfp_only.takeResidues(non_standard_index)
nonstd.writePdb('raw_nonstd.pdb')

#merge nonstd residues for total protein
for i in range(len(non_standard_index)-1):
    gfp_only.mergeResidues(64, name='CHR')
gfp_only.renumberResidues(start=1)
gfp_only.writePdb(mod_pdb)

clean = B.PDBCleaner( nonstd )
capped_nonstd = clean.capTerminals( nonstd, 0 )

#merge nonstd residues for nonstd part (capping code doesn't work if nonstd residues present)
for i in range(len(non_standard_index)-1):
    capped_nonstd.mergeResidues(1,name='CHR')
capped_nonstd.renumberResidues(start=64)
capped_nonstd.writePdb('raw_capped_nonstd.pdb')
                       
#protonation that ignores the residue names work so can use babel to protonate
#WARNING Babel splits the capping residues into new residues and distributes them through out the file,
# this might be a problem in some instances - red tools doesn't seem to mind though and -out.pdb it generates puts the capping residues back where they should be 
#should manually edit to make sure the caps and protons are in a sensible place/bond orders are correct)
mol = readfile("pdb", "raw_capped_nonstd.pdb").next()
mol.addh()
mol.write('pdb', 'capped_nonstd.pdb', overwrite=True)


#protonate gfp_only except for non_standard (pdb2pqr)
os.system('pdb2pqr --ff=amber --ffout=amber --nodebump {i} {o}'.format(i=mod_pdb, o= prot_mod_pqr))
os.system('babel {i} {o}'.format(i=prot_mod_pqr, o=prot_mod_pdb))

#Run Ante_red on capped_nonstd generating capped_nonstd-out.p2n
os.system('Ante_RED-1.5.pl {pdb_f}'.format(pdb_f='capped_nonstd.pdb'))

#mod .p2n to constrain charges of caps to 0
capped_nonstd = B.PDBModel('capped_nonstd-out.pdb')
ace_capping_atoms = [str(n+1) for n,r in enumerate(capped_nonstd.atoms['residue_name']) if r == 'ACE']
nme_capping_atoms = [str(n+1) for n,r in enumerate(capped_nonstd.atoms['residue_name']) if r == 'NME']

ace_remark = 'REMARK INTRA-MCC 0.0 |  ' + '  '.join(ace_capping_atoms) + ' | Remove\n'
nme_remark = 'REMARK INTRA-MCC 0.0 |  ' + '  '.join(nme_capping_atoms) + ' | Remove\n'

with open('capped_nonstd-out.p2n') as p2n_f:
    p2n_content = p2n_f.read()
split_contents = p2n_content.split('REMARK\n')

split_contents[-2] += ace_remark
split_contents[-2] += nme_remark
p2n_content = 'REMARK\n'.join(split_contents)

with open('mod_capped_nonstd-out.p2n', 'w') as p2n_f:
    p2n_f.write(p2n_content)


##use red (opt=true, qm program = gaussian) to generate mol2 file with partial charges
#run_red(mod_capped_nonstd-out.p2n)

    
#no nonstd residue present in file generated from pdb2pqr so now we add in the nonstd residue
#get coords of the protonated residue from the red_tools pdb output (babel mixes up the 
#residues when it protonates but Ante_red fixes this)
#get amber atom names from the mol2 file we are using to make the library file 
#(cannot take coords as red/gaussian end up shifting the zero coordinates)

os.system('antechamber -fi mol2 -i Mol_m1-o1-sm.mol2 -fo pdb -o mod_capped_nonstd.pdb')
amber_atom_names = B.PDBModel('mod_capped_nonstd.pdb').atoms['name']
gfp_chr = B.PDBModel('capped_nonstd-out.pdb').takeResidues([1])
gfp_chr.atoms.set('name', amber_atom_names)

prot_gfp = B.PDBModel(prot_mod_pdb)
#residues before the non_std residue
c0_res_list = range(len(prot_gfp.resList()))[0:non_standard_index[0]]
#residues after the non_std residue
c1_res_list = range(len(prot_gfp.resList()))[non_standard_index[0]:]
prot_gfp_c0 = prot_gfp.takeResidues(c0_res_list)
prot_gfp_c1 = prot_gfp.takeResidues(c1_res_list)


#build the master from the protonated standard residues and the non-standard residue (currently nonstd residue is maximally protonated - need to run propka/pdb2pqr with custom data for nonstd residue to have ph dependent protonation of nonstd residue
master_gfp = prot_gfp_c0.concat(gfp_chr).concat(prot_gfp_c1)
master_gfp.mergeChains(0)
master_gfp.mergeChains(0)
master_gfp.renumberResidues()
master_gfp.writePdb(master_pdb)
#antechamber renumbers the atoms so they are consistent (bizkit doesn't - leaves atom numbers inconsistent after the concatenation)
os.system('antechamber -fi pdb -i {m} -fo pdb -o {m}'.format(m=master_pdb))

#generate xyz
os.system('babel -i pdb {m1} -o xyz {m2}'.format(m1=master_pdb,m2=master_xyz))

#create set of complete list of gaff and amber atom types so we can map one to the other
os.system('antechamber -fi pdb -i {p} -fo mol2 -o {m}'.format(p=master_pdb, m=pdb_code +'_master_gaff.mol2'))
os.system('antechamber -fi pdb -i {p} -fo mol2 -o {m} -at amber'.format(p=master_pdb, m=pdb_code +'_master_amber.mol2'))
non_std_res = non_standard_index[0]
non_std_atom_indexes = [a['serial_number']-1 for a in B.PDBModel(master_pdb).resList()[non_std_res]]

#schrodinger suite utilities structconvert pdb -> mdf, computes connectivity first using standard AAs then standard heuristics
#then babel mdf -> sdf, sdf -> molmod -> neighbours
#master_sdf = SDFReader(pdb_code + '_master.sdf').next()
master_xyz = XYZReader(pdb_code + '_master.xyz').get_first_molecule()
master_xyz.set_default_graph()

#to determine the head and tail atoms of the non_standard residue we use the nearest neighbours
head_atom_ind, tail_atom_ind = get_ht_ind(master_xyz, non_std_atom_indexes)
head_atom_nm, tail_atom_nm = gfp_chr[head_atom_ind]['name'], gfp_chr[tail_atom_ind]['name']

#antechamber mol2 -> mol2 to generate a mol2 file that uses amber names fails because for the backbone N and C 
#because we have cut bonds and antechamber assumes that means they are double bonded! 
#so we need to take the names from the full protein
os.system('antechamber -fi mol2 -i Mol_m1-o1-sm.mol2 -fo mol2 -o amb_Mol_m1-o1-sm.mol2')
set_frag_atom_types('amb_Mol_m1-o1-sm.mol2', 'mod_capped_nonstd.mol2', master_pdb, atom_types='amber')
set_nonstd_name('mod_capped_nonstd.mol2', 'CHR')
#use parmchk2 on mol2 to generate .frcmod file
os.system('parmchk2 -a Y -f mol2 -i mod_capped_nonstd.mol2 -o mod_capped_nonstd.frcmod')

#if we use gaff atoms this defines a gaff forcefield for the non standard residue and specifies gaff atom 
#types along with the parameters defining their interaction however the rest of the protein is defined by 
#an amber forcefield so we need to define parameters that define the interaction between the terminal n and 
#c gaff atoms and the amber atom types that it can join too. To do this we can use the standard amber parameters. 

#gaff_atoms, gaff_bonds = mol2_parse(pdb_code +'_master_gaff.mol2')
#amber_atoms, amber_bonds = mol2_parse(pdb_code +'_master_amber.mol2')
#get bonds,angles and dihedrals required for atoms spanning the nonstandard residues and the standard residues
#master_lookup = get_bad(master_xyz, gaff_atoms, amber_atoms, non_std_atom_indexes)
#bonds, angles, dihedrals = master_lookup['mixed'] 
#amber_bonds, amber_angles, amber_dihedrals = master_lookup['amber']
#extra_params = get_amber_params([amber_bonds, amber_angles, amber_dihedrals)
#add_params('mod_capped_nonstd.frcmod', extra_params)

"""Could not find bond parameter for: C - n
Could not find bond parameter for: c - N
Building angle parameters.
Could not find angle parameter: O - C - n
Could not find angle parameter: C - n - hn
Could not find angle parameter: C - n - c3
Could not find angle parameter: CT - C - n
Could not find angle parameter: o - c - N
Could not find angle parameter: c - N - H
Could not find angle parameter: c - N - CT
Could not find angle parameter: c3 - c - N
Building proper torsion parameters.
 ** No torsion terms for  O-C-n-hn
 ** No torsion terms for  O-C-n-c3
 ** No torsion terms for  CT-C-n-hn
 ** No torsion terms for  CT-C-n-c3
 ** No torsion terms for  o-c-N-H
 ** No torsion terms for  o-c-N-CT
 ** No torsion terms for  c3-c-N-H
 ** No torsion terms for  c3-c-N-CT"""



#generate non_std.in and master.in
#run tleap using non_std.in to generate library, file
#need to use oldff for ff99SB in later version of amber

#set CHR.1 connect0 CHR.1.{head_atom}
#set CHR.1 connect1 CHR.1.{tail_atom}
#set CHR.1 restype protein
#set CHR.1 name "CHR"

tleap_1_content = """source oldff/leaprc.ff99SB
source leaprc.gaff
loadamberparams mod_capped_nonstd.frcmod
CHR = loadmol2 mod_capped_nonstd.mol2 
set CHR head CHR.1.{head_atom} 
set CHR tail CHR.1.{tail_atom}
set CHR name "CHR"
check CHR 
saveoff CHR CHR.lib 
saveamberparm CHR chr.prmtop chr.inpcrd
quit """.format(head_atom=head_atom_nm, tail_atom=tail_atom_nm)

with open('non_std.in', 'w') as tleap_in_f:
    tleap_in_f.write(tleap_1_content)

os.system('tleap -f {inp_fn}'.format(inp_fn='non_std.in'))

#run tleap using master.in to generate final params for gfp_master
#need to use oldff for ff99SB in later version of amber
tleap_2_content = """source oldff/leaprc.ff99SB
source leaprc.gaff 
loadamberparams mod_capped_nonstd.frcmod 
loadoff CHR.lib 
complex = loadpdb {mast}
check complex
saveamberparm complex 1GFL_chr.prmtop 1GFL_chr.inpcrd
savepdb complex 1GFL_chr.pdb 
quit """.format(mast=master_pdb)

with open(pdb_code + '.in', 'w') as tleap_in_f:
    tleap_in_f.write(tleap_2_content)

os.system('tleap -f {inp_fn}'.format(inp_fn=pdb_code + '.in'))

<IPython.core.display.Javascript object>

Capping N-terminal of chain 0 with ACE



Warning (ignored): Profile biomol skipped during concat.


Capping C-terminal of chain 0 with NME.


0

In [27]:
B.PDBModel(master_pdb).resList()[non_std_res]

[CrossView{'name': 'N', 'residue_number': 65, 'insertion_code': '', 'alternate': '', 'name_original': ' N  ', 'chain_id': '', 'occupancy': 0.0, 'element': 'N', 'segment_id': '', 'charge': '', 'residue_name': 'CHR', 'after_ter': 0, 'serial_number': 958, 'type': 'ATOM', 'temperature_factor': 0.0},
 CrossView{'name': 'H', 'residue_number': 65, 'insertion_code': '', 'alternate': '', 'name_original': ' H  ', 'chain_id': '', 'occupancy': 0.0, 'element': 'H', 'segment_id': '', 'charge': '', 'residue_name': 'CHR', 'after_ter': 0, 'serial_number': 959, 'type': 'ATOM', 'temperature_factor': 0.0},
 CrossView{'name': 'CA', 'residue_number': 65, 'insertion_code': '', 'alternate': '', 'name_original': ' CA ', 'chain_id': '', 'occupancy': 0.0, 'element': 'C', 'segment_id': '', 'charge': '', 'residue_name': 'CHR', 'after_ter': 0, 'serial_number': 960, 'type': 'ATOM', 'temperature_factor': 0.0},
 CrossView{'name': 'HA', 'residue_number': 65, 'insertion_code': '', 'alternate': '', 'name_original': ' HA 

In [24]:
get_ht_ind(master_xyz, non_std_atom_indexes)

array([ 0, 32])

In [26]:
non_std_atom_indexes

[957,
 958,
 959,
 960,
 961,
 962,
 963,
 964,
 965,
 966,
 967,
 968,
 969,
 970,
 971,
 972,
 973,
 974,
 975,
 976,
 977,
 978,
 979,
 980,
 981,
 982,
 983,
 984,
 985,
 986,
 987,
 988,
 989,
 990]

In [8]:
#test params

import simtk.unit as u
import simtk.openmm as mm
import mdtraj as md
import numpy as np
from mdtraj.reporters import NetCDFReporter
from simtk.openmm import app
from simtk.unit.quantity import Quantity
from sys import stdout

from pybel import readfile
from openmoltools import amber
from openmoltools.amber_parser import AmberParser

prmtop = app.AmberPrmtopFile('1GFL_chr.prmtop')
inpcrd = app.AmberInpcrdFile('1GFL_chr.inpcrd')
system = prmtop.createSystem(nonbondedMethod=app.CutoffNonPeriodic, 
                             nonbondedCutoff=1*u.nanometer, 
                             constraints=app.HBonds)

integrator = mm.LangevinIntegrator(300*u.kelvin, 1/u.picosecond, 0.002*u.picoseconds)
#platform = mm.Platform.getPlatformByName('CUDA')
#prop = dict(CudaPrecision='mixed') # Use mixed single/double precision
simulation = app.Simulation(prmtop.topology, system, integrator)#, platform, prop)
simulation.context.setPositions(inpcrd.positions)

if inpcrd.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

simulation.minimizeEnergy()
simulation.reporters.append(app.PDBReporter('output.pdb', 100))
simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True, potentialEnergy=True, temperature=True))
simulation.step(1000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
100,7865.05450226,55.1295876626
200,9559.53864433,103.710788983
300,10975.9279326,139.548264377
400,12107.3048806,169.514045903
500,13132.5670764,196.72691888
600,13998.8186401,214.318965602
700,14461.00167,231.978671791
800,15022.8940727,243.390605187
900,15269.4356357,259.542331995
1000,15636.437357,260.167472789


In [9]:
pdb_code = '1GFL'
orig_pdb = pdb_code + '.pdb'
mod_pdb = 'mod_' + orig_pdb
prot_mod_pdb = 'prot_' + mod_pdb
prot_mod_pqr = prot_mod_pdb.replace('.pdb', '.pqr')
master_pdb = pdb_code + '_master.pdb'

In [4]:
import os
#have conda install ambermini which includes the parameter files we want
os.environ['AMBERHOME']="/home/clyde/anaconda/envs/numpy17"

In [6]:
#gaussian needs types, partial charges, pdb name, residue name, residue id, and coords
# N-N3-0.184900(PDBName=N,ResName=SER,ResNum=1)     13.66700000   22.25500000  106.43400000 L

from cc_utils.chem_utils import get_amber_params_from_xml
from cc_utils.chem_utils import get_amber_params_from_dat
from cc_utils.chem_utils import get_amber_params_from_frcmod, write_gauss_amber_params, update_amber_params

#this currently doesn't inset the equivalent atom vdw values.

params = get_amber_params_from_dat('parm99')
sbmod_params=get_amber_params_from_frcmod('ff99SB')
ff99sb_params = update_amber_params(params, sbmod_params)

frcmod_fn = '/home/clyde/Dropbox/Project Stuff/Notebooks/oniom_bio/test/mod_capped_nonstd.frcmod'
nonstd_params = get_amber_params_from_frcmod(frcmod_fn)

#final_params = update_amber_params(get_amber_params_from_dat('parm96'), nonstd_params, overwrite=False)
final_params = update_amber_params(ff99sb_params, nonstd_params, overwrite=False)
final_param_str = write_gauss_amber_params(final_params)

param_str1 = write_gauss_amber_params(params)
param_str2 = write_gauss_amber_params(sbmod_params)
param_str3 = write_gauss_amber_params(nonstd_params)
param_str4 = write_gauss_amber_params(final_params)

with open('1GFL_frc.dat','w') as frc_f:
    frc_f.write(final_param_str) 
    
#Vale is very very beautiful with her Christams jumper

#pdb_code = '1GFL'
#pdb_fn = pdb_code + '_master.pdb'
#gfp_mol = ase_read_pdb(pdb_fn)
#atom_types, charges = get_amber_params(gfp_mol,lib_file)
#pdb_names,  res_names, res_nums = get_pdb_details(pdb_fn)

#amber_info = {'AtomTypes':atom_types, 'PartialCharges':charges,'PDBNames':pdb_names, 'ResNames':res_names, 'ResNums':res_nums, 'Param_str':final_param_str}

In [8]:
print(param_str1)

NonBon 3 1 0 0 0.000 0.000 0.500 0.000 0.000 -1.000

HrmStr1 OW HW  553.0  0.9572
HrmStr1 HW HW  553.0  1.5136
HrmStr1 C  C   310.0  1.525
HrmStr1 C  CA  469.0  1.409
HrmStr1 C  CB  447.0  1.419
HrmStr1 C  CM  410.0  1.444
HrmStr1 C  CT  317.0  1.522
HrmStr1 C  N   490.0  1.335
HrmStr1 C  N*  424.0  1.383
HrmStr1 C  NA  418.0  1.388
HrmStr1 C  NC  457.0  1.358
HrmStr1 C  O   570.0  1.229
HrmStr1 C  O2  656.0  1.250
HrmStr1 C  OH  450.0  1.364
HrmStr1 C  OS  450.0  1.323
HrmStr1 C  H4  367.0  1.080
HrmStr1 C  H5  367.0  1.080
HrmStr1 CA CA  469.0  1.400
HrmStr1 CA CB  469.0  1.404
HrmStr1 CA CM  427.0  1.433
HrmStr1 CA CN  469.0  1.400
HrmStr1 CA CT  317.0  1.510
HrmStr1 CA HA  367.0  1.080
HrmStr1 CA H4  367.0  1.080
HrmStr1 CA N2  481.0  1.340
HrmStr1 CA NA  427.0  1.381
HrmStr1 CA NC  483.0  1.339
HrmStr1 CA OH  450.0  1.364
HrmStr1 CB CB  520.0  1.370
HrmStr1 CB N*  436.0  1.374
HrmStr1 CB NB  414.0  1.391
HrmStr1 CB NC  461.0  1.354
HrmStr1 CD HA  367.0  1.080
HrmStr1 CD CD  469.0 

In [12]:
cd test

/home/clyde/Dropbox/Project Stuff/Notebooks/oniom_bio/test


In [13]:
from openmoltools.amber import find_executable
import os
full_path = find_executable("parmchk2")
parm_path = os.path.split(full_path)[0] + '/../dat/leap/parm/'

In [31]:
import openmoltools.amber_parser as ap
#by default amberparser skips water residues as we do not wish to we set skipClasses to empty
ap.skipClasses = []

amber_parser = ap.AmberParser()
amber_parser.process_dat_file(parm_path + 'parm96.dat')
#amber_parser.process_dat_file(parm_path + 'parm99.dat')
#amber_parser.process_frc_file(parm_path + 'frcmod.ff99SB')
#amber_parser.process_frc_file('mod_capped_nonstd.frcmod')
#amber_parser.process_mol2_file('1GFL_master_gaff.mol2')
##had to hack process_library_file_method
#amber_parser.process_library_file('CHR.lib')

In [26]:
amber_parser.residueConnections

{}

In [15]:
final_param_str.split('\n')[500]

'AmbTrs CT CT  N   C   0     0     0     0     2.00   2.00   0.40   0.00   1.0'

In [16]:
from lxml import etree
from io import StringIO
class Gaussian_MM(object):
    """
    Generates gaussian readable forcefield files from an amber_parser object
    
    Currently omits parameters for water
    """
    
    def __init__(self, amber_parser):
        self.amber_parser = amber_parser
    
    def parse_xml(self):

        xml = unicode(amber_parser.generate_xml().read())
        force_tree = etree.ElementTree().parse(StringIO(xml))
        atoms, residues, bonds, angles, master_torsions, nonbond = force_tree.getchildren()

        self.atoms = [a.attrib for a in atoms]
        self.residues = [r.attrib for r in residues]
        self.bonds = [b.attrib for b in bonds]
        self.angles = [a.attrib for a in angles]
        self.nonbond = [n.attrib for n in nonbond]
        self.proper_torsions = [t.attrib for t in master_torsions if t.tag == 'Proper']
        self.improper_torsions = [t.attrib for t in master_torsions if t.tag =='Improper']

        #sanity check
        assert(len(self.proper_torsions) + len(self.improper_torsions) == len(master_torsions))

        return
    
    def extract_bonds(self):
        bond_strs = []
        for bond in self.bonds:
            bond_data = bond['class1'], bond['class2'],\
                        float(bond['k'])/(200*4.184), float(bond['length'])*10,     
            bond_strs.append('HrmStr1 {:<2} {:<2}  {:<6} {:<5}'.format(*bond_data))
        return bond_strs
    
    def extract_angles(self):
        angle_strs = []
        for angle in self.angles:
            angle_data = angle['class1'], angle['class2'], angle['class3'],\
                         float(angle['k'])/(2*4.184), float(angle['angle'])*360/(2*np.pi)
            angle_strs.append('HrmBnd1 {:<2} {:<2}  {:<2}  {:6} {:6}'.format(*angle_data))
        return angle_strs
            
    def extract_torsions(self):
        """
        Extract parameters for Torsions
        
        NB amber parser does not record the raw magnitudes instead recording the magnitude/NPaths
           since the final equation involves magnitude/NPATH replacing the magnitude with the scaled
           magnitude and setting NPaths to 1.0 should leave the forcefield unchanged. We adopt
           this strategy here.
        """
        
        torsion_strs = []
        for torsion in self.proper_torsions:
            atoms = [torsion['class1'], torsion['class2'], torsion['class3'], torsion['class4']]
            for i,a in enumerate(atoms):
                if not atoms[i] :
                    atoms[i] = '*'
                    
            periodicity = torsion.get('periodicity1',None), torsion.get('periodicity2',None),\
                          torsion.get('periodicity3',None), torsion.get('periodicity4',None)
            periodicity = [p for p in periodicity if p]
            periodicity = np.array(periodicity, dtype=np.int32)-1

            temp_phases = torsion.get('phase1',None), torsion.get('phase2',None),\
                          torsion.get('phase3',None), torsion.get('phase4',None)
            temp_phases = [ph for ph in temp_phases if ph]
            temp_phases = np.array(temp_phases, dtype=np.float64) *360/(2*np.pi)
            phases = np.zeros(4)
            phases[periodicity] = temp_phases
            
            temp_mags = torsion.get('k1',None), torsion.get('k2',None),\
                        torsion.get('k3',None), torsion.get('k4',None)
            temp_mags = np.array(temp_mags, dtype=np.float64)/4.184
            mags = np.zeros(4)
            mags[periodicity] = temp_mags

            combi = atoms + list(phases) + list(mags)
            torsion_strs.append('AmbTrs {:<2} {:<3} {:<3} {:<3} {:<5.0f} {:<5.0f} {:<5.0f} {:<5.0f} {:<6.3f} {:<6.3f} {:<6.3f} {:<6.3f} 1.0'.format(*combi))
        
        #AmberParser records torsions in the opposite order they are found in the original dat file.
        #To make comparisons with the original file and the prm files included with gaussian we use the original
        #order
        torsion_strs = list(reversed(torsion_strs))
        return torsion_strs 
    
    def extract_improper(self):
        improper_strs = []
        
        for improper in self.improper_torsions:
            #AmberParser reorders these for some reason
            improper_data = [improper['class2'], improper['class3'], improper['class1'], improper['class4'],\
                             float(improper['k1'])/4.184, improper['periodicity1']]

            if improper_data[0] == '':
                improper_data[0] = '*'
            if improper_data[1] == '':
                improper_data[1] = '*'
            if improper_data[2] == '':
                improper_data[2] = '*'
            if improper_data[3] == '':
                improper_data[3] = '*'

            improper_strs.append('ImpTrs {:<2} {:<2} {:<3} {:<3} {:<6} 180.0  {:<5}'.format(*improper_data))
        
        #AmberParser records Improper torsions in the opposite order they are found in the original dat file.
        #To make comparisons with the original file and the prm files included with gaussian we use the original
        #order
        improper_strs = list(reversed(improper_strs))
        return improper_strs

    def extract_vdw(self):
        vdw_strs = []
        for a in  self.amber_parser.vdw:
            vdw_data = a, float(self.amber_parser.vdw[a][0]), float(self.amber_parser.vdw[a][1])
            vdw_strs.append('VDW {:<2} {:<9.9f} {:<9.9f}'.format(*vdw_data))
        
        for a in self.amber_parser.vdwEquivalents:
            equiv_a = self.amber_parser.vdwEquivalents[a]
            vdw_data = a, float(self.amber_parser.vdw[equiv_a][0]), float(self.amber_parser.vdw[equiv_a][1]) 
            vdw_strs.append('VDW {:<2} {:<9.9f} {:<9.9f}'.format(*vdw_data))

        #amber_parser stores the vdw files in a dictionary so there is no way to preserve the original data file
        #ordering as such we simply sort
        return sorted(vdw_strs)
    
    def generate_prm(self):
        init_prm = ['!',
                    '! Non-bonded interaction',
                    '!',
                    'NonBon 3 1 0 0 0.000 0.000 0.500 0.000 0.000 -1.000']
        
        bond_prm = ['!',
                   '! Stretches',
                   '!'] + self.extract_bonds()
        
        angles_prm = [ '!',
                       '! Angles',
                       '!'] + self.extract_angles()
        
        torsions_prm = ['!',
                        '! Torsions',
                        '!',] + self.extract_torsions()
        
        improper_prm = [ '!',
                         '! Improper torsions',
                         '!',] + self.extract_improper()
        
        vdw_prm = ['!',
                   '! Vanderwaals parameters',
                   '!',] + self.extract_vdw()
        
        return '\n'.join(init_prm + bond_prm + angles_prm + torsions_prm + improper_prm + vdw_prm)

In [32]:
g_mm = Gaussian_MM(amber_parser)
g_mm.parse_xml()
g_mm.extract_torsions()
g_mm.extract_angles()
g_mm.extract_angles()
g_mm.extract_improper()
g_mm.generate_prm().split('\n')

['!',
 '! Non-bonded interaction',
 '!',
 'NonBon 3 1 0 0 0.000 0.000 0.500 0.000 0.000 -1.000',
 '!',
 '! Stretches',
 '!',
 'HrmStr1 OW HW  553.0  0.9572',
 'HrmStr1 HW HW  553.0  1.5136',
 'HrmStr1 C  CA  469.0  1.409',
 'HrmStr1 C  CB  447.0  1.419',
 'HrmStr1 C  CM  410.0  1.444',
 'HrmStr1 C  CT  317.0  1.522',
 'HrmStr1 C  N*  424.0  1.383',
 'HrmStr1 C  NA  418.0  1.388',
 'HrmStr1 C  NC  457.0  1.358',
 'HrmStr1 C  O   570.0  1.229',
 'HrmStr1 C  O2  656.0  1.25 ',
 'HrmStr1 C  OH  450.0  1.364',
 'HrmStr1 CA CA  469.0  1.4  ',
 'HrmStr1 CA CB  469.0  1.404',
 'HrmStr1 CA CM  427.0  1.433',
 'HrmStr1 CA CT  317.0  1.51 ',
 'HrmStr1 CA HA  367.0  1.08 ',
 'HrmStr1 CA H4  367.0  1.08 ',
 'HrmStr1 CA N2  481.0  1.34 ',
 'HrmStr1 CA NA  427.0  1.381',
 'HrmStr1 CA NC  483.0  1.339',
 'HrmStr1 CB CB  520.0  1.37 ',
 'HrmStr1 CB N*  436.0  1.374',
 'HrmStr1 CB NB  414.0  1.391',
 'HrmStr1 CB NC  461.0  1.354',
 'HrmStr1 CK H5  367.0  1.08 ',
 'HrmStr1 CK N*  440.0  1.371',
 'HrmStr1

In [18]:
g_mm.amber_parser.generate_xml().readlines()

['<!-- Time and parameters of origin: -->\n',
 '<!-- 2015-08-31 12:45:01.173298 -->\n',
 '<!--  f /home/clyde/.ipython/profile_default/security/kernel d45c9e88 e84e 468d 9a10 b16fd9261c1f.json   profile dir /home/clyde/.ipython/profile_default -->\n',
 '\n',
 '<ForceField>\n',
 ' <AtomTypes>\n',
 ' </AtomTypes>\n',
 ' <Residues>\n',
 ' </Residues>\n',
 ' <HarmonicBondForce>\n',
 '  <Bond class1="C" class2="CA" length="0.1409" k="392459.2"/>\n',
 '  <Bond class1="C" class2="CB" length="0.1419" k="374049.6"/>\n',
 '  <Bond class1="C" class2="CM" length="0.1444" k="343088.0"/>\n',
 '  <Bond class1="C" class2="CT" length="0.1522" k="265265.6"/>\n',
 '  <Bond class1="C" class2="N*" length="0.1383" k="354803.2"/>\n',
 '  <Bond class1="C" class2="NA" length="0.1388" k="349782.4"/>\n',
 '  <Bond class1="C" class2="NC" length="0.1358" k="382417.6"/>\n',
 '  <Bond class1="C" class2="O" length="0.1229" k="476976.0"/>\n',
 '  <Bond class1="C" class2="O2" length="0.125" k="548940.8"/>\n',
 '  <Bond

In [222]:
def check_param_str():
    return

def check_gmm_param_str():
    return

In [156]:
amb96

['!',
 '! Non-bonded interaction',
 '!',
 'NonBon 3 1 0 0 0.000 0.000 0.500 0.000 0.000 -1.000',
 '!',
 '! Stretches',
 '!',
 'HrmStr1 OW HW  553.0  0.9572',
 'HrmStr1 HW HW  553.0  1.5136',
 'HrmStr1 C  CA  469.0  1.409 ',
 'HrmStr1 C  CB  447.0  1.419 ',
 'HrmStr1 C  CM  410.0  1.444 ',
 'HrmStr1 C  CT  317.0  1.522 ',
 'HrmStr1 C  N*  424.0  1.383 ',
 'HrmStr1 C  NA  418.0  1.388 ',
 'HrmStr1 C  NC  457.0  1.358 ',
 'HrmStr1 C  O   570.0  1.229 ',
 'HrmStr1 C  O2  656.0  1.250 ',
 'HrmStr1 C  OH  450.0  1.364 ',
 'HrmStr1 CA CA  469.0  1.400 ',
 'HrmStr1 CA CB  469.0  1.404 ',
 'HrmStr1 CA CM  427.0  1.433 ',
 'HrmStr1 CA CT  317.0  1.510 ',
 'HrmStr1 CA HA  367.0  1.080 ',
 'HrmStr1 CA H4  367.0  1.080 ',
 'HrmStr1 CA N2  481.0  1.340 ',
 'HrmStr1 CA NA  427.0  1.381 ',
 'HrmStr1 CA NC  483.0  1.339 ',
 'HrmStr1 CB CB  520.0  1.370 ',
 'HrmStr1 CB N*  436.0  1.374 ',
 'HrmStr1 CB NB  414.0  1.391 ',
 'HrmStr1 CB NC  461.0  1.354 ',
 'HrmStr1 CK H5  367.0  1.080 ',
 'HrmStr1 CK N*  

In [146]:
#amber 96 param file included with Gaussian
with open('../amber.prm') as f:
    amb96 = f.read().split('\n')
    
test_amb96 = write_gauss_amber_params(get_amber_params_from_dat('parm96')).split('\n')
test2_amb96

In [226]:
test_amb96

['NonBon 3 1 0 0 0.000 0.000 0.500 0.000 0.000 -1.000',
 '',
 'HrmStr1 OW HW  553.0  0.9572',
 'HrmStr1 HW HW  553.0  1.5136',
 'HrmStr1 C  CA  469.0  1.409',
 'HrmStr1 C  CB  447.0  1.419',
 'HrmStr1 C  CM  410.0  1.444',
 'HrmStr1 C  CT  317.0  1.522',
 'HrmStr1 C  N*  424.0  1.383',
 'HrmStr1 C  NA  418.0  1.388',
 'HrmStr1 C  NC  457.0  1.358',
 'HrmStr1 C  O   570.0  1.229',
 'HrmStr1 C  O2  656.0  1.250',
 'HrmStr1 C  OH  450.0  1.364',
 'HrmStr1 CA CA  469.0  1.400',
 'HrmStr1 CA CB  469.0  1.404',
 'HrmStr1 CA CM  427.0  1.433',
 'HrmStr1 CA CT  317.0  1.510',
 'HrmStr1 CA HA  367.0  1.080',
 'HrmStr1 CA H4  367.0  1.080',
 'HrmStr1 CA N2  481.0  1.340',
 'HrmStr1 CA NA  427.0  1.381',
 'HrmStr1 CA NC  483.0  1.339',
 'HrmStr1 CB CB  520.0  1.370',
 'HrmStr1 CB N*  436.0  1.374',
 'HrmStr1 CB NB  414.0  1.391',
 'HrmStr1 CB NC  461.0  1.354',
 'HrmStr1 CK H5  367.0  1.080',
 'HrmStr1 CK N*  440.0  1.371',
 'HrmStr1 CK NB  529.0  1.304',
 'HrmStr1 CM CM  549.0  1.350',
 'HrmStr1

In [147]:
test_b = test_amb96[2:85]
b = [l.strip() for l in amb96[7:90]]

test_a = test_amb96[86:277]
a = [l.strip() for l in amb96[93:284]]

[l.split() for l in a] == [l.split() for l in test_a]

test_t = test_amb96[278:344]
t = [l.strip() for l in amb96[287:346]]

#[l.split() for l in t], [l.split() for l in test_t]

In [153]:
len(g_mm.extract_bonds()), len(test_b)

(81, 83)

In [154]:
f=1.234
'{:<6.2f}'.format(f)

'1.23  '

In [176]:
[t for t in amber_parser.torsions if t[:4] == ['CT','CT','N','C']][-1]

['CT',
 'CT',
 'N',
 'C',
 0.0,
 '0.0',
 4,
 0.4,
 '0.0',
 3,
 2.0,
 '0.0',
 2,
 2.0,
 '0.0',
 1]

In [90]:
l='X -C -C -X    4   14.50        180.0             2.         Junmei et al, 1999\n'
line=l.strip()
fields = line[11:].split()
fields
periodicity = int(float(fields[3]))
[float(fields[1]) / float(fields[0]), fields[2], abs(periodicity)]

[3.625, '180.0', 2]

In [92]:
periodicity

2

In [91]:
[line[:2].strip(), line[3:5].strip(), line[6:8].strip(), line[9:11].strip(), float(fields[1]) / float(fields[0]), fields[2], abs(periodicity)]

['X', 'C', 'C', 'X', 3.625, '180.0', 2]

In [186]:
len([l for l in final_param_str.split('\n') if 'AmbTrs' in l]), len([l for l in x if 'Proper' in l])

(169, 169)

In [ ]:
from chem_utils import *
from molmod.io import XYZReader

non_standard_index = [64,65,66]
non_std_res = non_standard_index[0]
non_std_atom_indexes = [a['serial_number']-1 for a in B.PDBModel(master_pdb).resList()[non_std_res]]

master_xyz = XYZReader(pdb_code + '_master.xyz').get_first_molecule()
master_xyz.set_default_graph()

gaff_atoms, gaff_bonds = mol2_parse(pdb_code +'_master_gaff.mol2')
amber_atoms, amber_bonds = mol2_parse(pdb_code +'_master_amber.mol2')
#get bonds,angles and dihedrals required for atoms spanning the nonstandard residues and the standard residues
master_lookup = get_bad(master_xyz, gaff_atoms, amber_atoms, non_std_atom_indexes)
bonds, angles, dihedrals = master_lookup['mixed'] 
amber_bonds, amber_angles, amber_dihedrals = master_lookup['amber']
#extra_params = get_amber_params([amber_bonds, amber_angles, amber_dihedrals)
#add_params('mod_capped_nonstd.frcmod', extra_params)

In [ ]:
amber_dihedrals

In [ ]:

#for i,atom in enumerate(protein):
#    atom.flag = amber_atoms[i].symbol
#    atom.charge = amber_atoms[i].charge
#protein.set_calculator(Gaussian(label='test_oniom_bio',method='oniom'))
#protein.calc.set_ff_params(gaus_amber_params)


get_partial_charges(mol)
get_pdb_names(pdb)
get

In [ ]:
def get_amber_info(pdb_fn):
    """Reads a PDB and extracts the parameters needed for using amber on the protein"""
    parser = Bio.PDB.PDBParser()
    protein=parser.get_structure(pdb_fn.replace('.pdb',''), pdb_fn)
    protein_atoms = [a for a in protein.get_atoms()]
    
    pdb_names = [a.name for a in protein_atoms]
    pdb_resnames = [a.parent.resname for a in protein_atoms]
    pdb_resnums = [a.parent.id[1] for a in protein_atoms]
    atom_types = 

In [ ]:
import Bio
pdb_fn = '1GFL_master.pdb'
parser = Bio.PDB.PDBParser()
protein=parser.get_structure(pdb_fn.replace('.pdb',''), pdb_fn)
protein_atoms = [a for a in protein.get_atoms

In [ ]:
a=protein_atoms[1]
residues = [r for r in protein.get_residues()]
r=residues[10].id[1]
r

In [ ]:
#frcmod_fn, dat_param_path = os.path.split(vers)
#dat_param_path
#print(write_gauss_amber_params())

dict_in_dict = lambda d1,d2: all(item in d2.items() for item in d1.items())

frcmod_elements = [b['element'] for b in non_std_params['types']]
parm_elements = [b['element'] for b in parm96_params['types']]

frcmod_pairs = [[b['t1'],b['t2']] for b in non_std_params['bondLengths']]
parm_pairs = [[b['t1'],b['t2']] for b in parm96_params['bondLengths']]
rev_parm_pairs = [list(reversed(p)) for p in parm_pairs]

frcmod_triples = [[b['t1'],b['t2'],b['t3']] for b in non_std_params['bondAngles']]
parm_triples = [[b['t1'],b['t2'],b['t3']] for b in parm96_params['bondAngles']]
rev_parm_triples = [list(reversed(p)) for p in parm_triples]

frcmod_quads = [[b['t1'],b['t2'],b['t3'],b['t4']] for b in non_std_params['bondTorsions']]
parm_quads = [[b['t1'],b['t2'],b['t3'],b['t4']] for b in parm96_params['bondTorsions']]
rev_parm_quads = [list(reversed(p)) for p in parm_quads]

frcmod_iquads = [[b['t1'],b['t2'],b['t3'],b['t4']] for b in non_std_params['bondImpropers']]
parm_iquads = [[b['t1'],b['t2'],b['t3'],b['t4']] for b in parm96_params['bondImpropers']]
rev_parm_iquads = [list(reversed(p)) for p in parm_iquads]

new_elements = [p for p in frcmod_elements if p not in parm_elements]
new_pairs = [p for p in frcmod_pairs if p not in parm_pairs and p not in rev_parm_pairs]
new_triples = [p for p in frcmod_triples if p not in parm_triples and p not in rev_parm_triples]
new_quads = [p for p in frcmod_quads if p not in parm_quads and p not in rev_parm_quads]
new_iquads = [p for p in frcmod_iquads if p not in parm_iquads and p not in rev_parm_iquads]

reduced_non_std_params = {'types':[],'bondLengths':[],'bondAngles':[], 'bondTorsions':[], 'bondImpropers':[]}

reduced_non_std_params['types'] = [e for e in non_std_params['types'] if e['element'] in new_elements]
reduced_non_std_params['bondLengths'] = [e for e in non_std_params['bondLengths'] if [e['t1'],e['t2']] in new_pairs]
reduced_non_std_params['bondAngles'] = [e for e in non_std_params['bondAngles'] if [e['t1'],e['t2'],e['t3']] in new_triples]
reduced_non_std_params['bondTorsions'] = [e for e in non_std_params['bondTorsions'] if [e['t1'],e['t2'],e['t3'],e['t4']] in new_quads]
reduced_non_std_params['bondImpropers'] = [e for e in non_std_params['bondImpropers'] if [e['t1'],e['t2'],e['t3'],e['t4']] in new_iquads]

In [ ]:
new_pairs, new_triples, new_quads, new_iquads

In [ ]:
{'a':1,'b':2}.items()

In [ ]:
{'a':1,'b':2,'c':3}.items()

In [ ]:
def sum_dihedrals(unsummed_dihedrals):
    for torsion in unsummed_dihedrals:
        comp=abs(int(torsion.pop('Nt').split('.')[0]))
        torsion.update({'gamma_{c}'.format(c=comp):torsion.pop('gamma')})
        torsion.update({'Vn_{c}'.format(c=comp):torsion.pop('Vn')})
    
    unique_dihedrals = set([(t['t1'],t['t2'],t['t3'],t['t4']) for t in unsummed_dihedrals])
    for unique_dihedral in unique_dihedrals:
        torsion = next(t for t in unsummed_dihedrals if t and (t['t1'], t['t2'], t['t3'], t['t4']) == unique_dihedral)
        torsion_inds = [i for i,t in enumerate(unsummed_dihedrals) if t and (t['t1'], t['t2'], t['t3'], t['t4']) == unique_dihedral]
    
        for torsion_ind in torsion_inds:
            torsion_2 = unsummed_dihedrals[torsion_ind]
            if (torsion['t1'], torsion['t2'], torsion['t3'], torsion['t4']) == (torsion_2['t1'], torsion_2['t2'], torsion_2['t3'], torsion_2['t4']) and torsion is not torsion_2:
                torsion.update(torsion_2)
                unsummed_dihedrals[torsion_ind] = None
    return [t for t in unsummed_dihedrals if t]


def gen_dihedral_str(torsion):
    if torsion['t1'] == 'X':
        torsion['t1'] = '*'
    if torsion['t2'] == 'X':
        torsion['t2'] = '*'
    if torsion['t3'] == 'X':
        torsion['t3'] = '*'
    if torsion['t4'] == 'X':
        torsion['t4'] = '*'

    torsion['Vn_1'] = torsion.get('Vn_1','0.00')
    torsion['Vn_2'] = torsion.get('Vn_2','0.00')
    torsion['Vn_3'] = torsion.get('Vn_3','0.00')
    torsion['Vn_4'] = torsion.get('Vn_4','0.00')
    torsion['gamma_1'] = str(int(float(torsion.get('gamma_1','0'))))
    torsion['gamma_2'] = str(int(float(torsion.get('gamma_2','0'))))
    torsion['gamma_3'] = str(int(float(torsion.get('gamma_3','0'))))
    torsion['gamma_4'] = str(int(float(torsion.get('gamma_4','0'))))
    

    return 'AmbTrs {a1:<2} {a2:<3} {a3:<3} {a4:<3} {g1:<5} {g2:<5} {g3:<5} {g4:<5} {v1:<6} {v2:<6} {v3:<6} {v4:<6} {p}.0'.format(a1=torsion['t1'], a2=torsion['t2'], a3=torsion['t3'], a4=torsion['t4'],
                                                                                                                                 g1=torsion['gamma_1'], g2=torsion['gamma_2'], g3=torsion['gamma_3'], g4=torsion['gamma_4'],
                                                                                                                                 v1=torsion['Vn_1'], v2=torsion['Vn_2'], v3=torsion['Vn_3'], v4=torsion['Vn_4'],
                                                                                                                                 p=torsion['npth'])

    
def gen_improper_str(improper):
    if improper['t1'] == 'X':
        improper['t1'] = '*'
    if improper['t2'] == 'X':
        improper['t2'] = '*'
    if improper['t3'] == 'X':
        improper['t3'] = '*'
    if improper['t4'] == 'X':
        improper['t4'] = '*'
        
    return 'ImpTrs {a1:<2} {a2:<2} {a3:<3} {a4:<3} {v:<6} {g:<5} {n}'.format(a1=improper['t1'], a2=improper['t2'], a3=improper['t3'], a4=improper['t4'],
                                                                             g=improper['gamma'], v=improper['Vn'], n=improper['Nt'])

stretches = []
for stretch in p['bondLengths']:
    stretches.append('HrmStr1 {a1:<2} {a2:<3} {k:<6} {bl}'.format(a1= stretch['t1'], a2 = stretch['t2'], k=stretch['keq'], bl=stretch['req']))
angles = []
for angle in p['bondAngles']:
    angles.append('HrmBnd1 {a1:<2} {a2:<3} {a3:<3} {k:<6} {ang}'.format(a1= angle['t1'], a2 = angle['t2'], a3 = angle['t3'], k=angle['keq'], ang=angle['req']))
torsions = []
p['bondTorsions'] = sum_dihedrals(p['bondTorsions'])
for torsion in p['bondTorsions']:
    torsions.append(gen_dihedral_str(torsion))
impropers = []
for improper in p['bondImpropers']:
    impropers.append(gen_improper_str(improper))  
vdws = []
for vdw in p['types']:
    vdws.append('VDW {a:<2} {r:<6} {d}'.format(a=vdw['name'], r=vdw['vdwRadius'], d=vdw['potentialWellDepth']))
nbnds = ['NonBon 3 1 0 0 0.000 0.000 0.500 0.000 0.000 -1.000']

In [ ]:
bonds,angles, dihedrals

In [ ]:
l[0]

In [ ]:
t

In [ ]:
orig_atoms[957:980]

In [ ]:
fragment_orig_atoms

In [ ]:
t=B.PDBModel('capped_nonstd-out.pdb')


In [ ]:
t.renameAmberRes()

In [ ]:
t.report()

In [ ]:
capped_nonstd.mergeResidues(1,name='CHR')

In [ ]:
capped_nonstd.atoms['residue_name']

In [ ]:
import scientific

In [ ]:
import pdb2pqr
import copy
from cc_notebook_utils import unwind
pdb_f=pdb2pqr.src.structures.getPDBFile('raw_capped_nonstd.pdb')
pdblist, errlist =pdb2pqr.src.pdb.readPDB(pdb_f)
pdb_f.close()
myDefinition = pdb2pqr.src.routines.Definition()
myProtein = pdb2pqr.src.protein.Protein(pdblist, myDefinition)
myRoutines = pdb2pqr.src.routines.Routines(myProtein, False)
residues = myProtein.getResidues()
atoms = unwind([r.atoms for r in residues])
coords1 = [a.getCoords() for a in atoms]
myRoutines.debumpResidue(residues[0],'')
residues = myProtein.getResidues()
atoms = unwind([r.atoms for r in residues])
coords2 = [a.getCoords() for a in atoms]

In [ ]:
coords1 == coords2

In [ ]:
capped_nonstd.compress( capped_nonstd.maskHeavy())

In [ ]:
import os
import Biskit as B

pdb_code = '1GFL'
orig_pdb = pdb_code + '.pdb'


non_standard_index = [64,65,66]

gfp_raw=B.PDBModel(orig_pdb)
gfp_only = gfp_raw.takeChains([0])


nonstd = gfp_only.takeResidues(non_standard_index)
nonstd.mergeResidues(0)
nonstd.mergeResidues(0)

clean = B.PDBCleaner( nonstd )
capped_nonstd = clean.capTerminals( nonstd, 0 )

In [ ]:
gfp_only = gfp_raw.takeChains([0])
gfp_only.mergeResidues(64)
gfp_only.mergeResidues(64)

nonstd_v2 = gfp_only.takeResidues(non_standard_index[0:1])

clean_2 = B.PDBCleaner( nonstd_v2 )
capped_nonstd_v2 = clean_2.capTerminals( nonstd_v2, 0 )

In [ ]:

for i in range(len(chromo.atoms['residue_name'])):
    chromo.atoms['residue_name'][i] = 'GFP'

In [ ]:
#protonation(non_standard)
#hydrogens(non_standard)
#get_new_res_names(non_standard)
#non_standard.xplor2amber()
#non_standard.writePdb('1GFL_non_standard.pdb')


In [ ]:
import Biskit as B

gfp = B.PDBModel('1GFL.pdb')
#gfp = B.PDBModel('1GFL')
chromo = gfp.takeResidues([64,65,66])
#del chromo.residues['biomol']   ## remove the bioUnit residue profile



In [ ]:
chromo, capped_chromo

In [ ]:
chromo.residues['biomol'][2:] += 0
chromo.residues['biomol']

In [ ]:
m2.resList()

In [ ]:
chromo.residues, m1.residues

In [ ]:
gfp_raw.residues.set

In [ ]:
chromo.info

In [ ]:
b=B.BioUnit.BioUnit

In [ ]:
T=m1.residues.concat(chromo.residues)

In [ ]:
gfp_raw.residues['biomol']

In [ ]:
m1.biounit.makeMultimer(0)

In [ ]:
m1.lenChains( breaks=True )

In [ ]:
gfp_raw.biounit.biomol[0][0]

In [ ]:
e_chain = m1.takeChains([])
t=chromo.concat(e_chain)
t.biounit

In [ ]:
B.PDBProfiles( m1,
                                       profiles=getattr(m1,'resProfiles',{}),
                                       infos=getattr(m1,'resProfiles_info',{}) )

In [ ]:
chromo.residues['biomol']

In [ ]:
B.PDBModel().residues['biomol']

In [ ]:
m1.biomodel

In [ ]:
gfp_raw.residues['biomol']

In [ ]:
chromo['temperature_factor']

In [ ]:
m1['temperature_factor']

In [ ]:
import Biskit as B

chromo = gfp_only.takeResidues([64,65])
del chromo.residues['biomol']   ## remove the bioUnit residue profile
#chromo.residues['biomol']
parm = B.AmberParmBuilder( chromo )
chromo = parm.capACE( chromo, 0 )
#m = parm.capNME( m, 0 )

In [ ]:
chromo=parm.capNME( chromo, 0 )

In [ ]:
parm.capNME( chromo, 0 )

In [ ]:
pdb_clean.unresolvedTerminals(chromophore)

In [ ]:
capN = pdb_clean.filterProteinChains(chromophore, [0], chromophore.chainIndex(breaks=False))
capC = pdb_clean.filterProteinChains(chromophore, [0], chromophore.chainEndIndex(breaks=False))

In [ ]:
import Biskit.tools as t
import Biskit.PDBModel as PDBModel

model = chromophore
chain = 0
breaks=False
F_ace_cap = t.dataRoot() + '/amber/leap/ace_cap.pdb'

c_start = model.chainIndex( breaks=breaks )
c_end = model.chainEndIndex( breaks=breaks)
Nterm_is_break = False
Cterm_is_break = False

if breaks:
    Nterm_is_break = c_start[chain] not in model.chainIndex()
    Cterm_is_break = c_end[chain] not in model.chainEndIndex()
    
m_ace = PDBModel( F_ace_cap )

chains_before = model.takeChains( range(chain), breaks=breaks )
m_chain       = model.takeChains( [chain], breaks=breaks )
chains_after  = model.takeChains( range(chain+1, len(c_start)),
                                  breaks=breaks )

m_term  = m_chain.resModels()[0]

## we need 3 atoms for superposition, CB might mess things up but
## could help if there is no HN
##        if 'HN' in m_term.atomNames():
m_ace.remove( ['CB'] )  ## use backbone 'C' rather than CB for fitting 

## rename overhanging residue in cap PDB
for a in m_ace:
    if a['residue_name'] != 'ACE':
        a['residue_name'] = m_term.atoms['residue_name'][0]
    else:
        a['residue_number'] = m_term.atoms['residue_number'][0]-1
        a['chain_id']       = m_term.atoms['chain_id'][0]
        a['segment_id']     = m_term.atoms['segment_id'][0]

## fit cap onto first residue of chain
m_ace = m_ace.magicFit( m_term )

cap = m_ace.resModels()[0]
serial = m_term['serial_number'][0] - len(cap)
cap['serial_number'] = range( serial, serial + len(cap) )

## concat cap on chain
m_chain = cap.concat( m_chain, newChain=False )

## re-assemble whole model
r = chains_before.concat( m_chain, newChain=not Nterm_is_break)
r = r.concat( chains_after, newChain=not Cterm_is_break)

In [ ]:
non_standard.biounit.keys()

In [ ]:
m_chain.writePdb('capped_chromo.pdb')

In [ ]:
from Biskit import PDBCleaner
pdb_clean = PDBCleaner(chromophore)
capped=pdb_clean.capTerminals(capN=[1])

In [ ]:
capped.writePdb('capped_chromo.pdb')

In [ ]:
#parm_build =Biskit.AmberParmBuilder(non_standard)
#parm_build.parmMirror('~/Temp/1gfl_amber_parms')
#gfp_mod = Biskit.PDBModel('1GFL.pdb')
#gfp_mod['chain_id']
#parm_build.capNME(gfp_raw,0)
#parm_build.capACE(non_standard,0)
#methylate(non_standard)
#get_connectivity(non_standard)
#get_atom_types(non_standard)
#get_partial_charges(non_standard)

In [ ]:
r1, r2 = non_standard.resModels()

In [ ]:
c=Biskit.PDBModel.concat(r1,r2)

In [ ]:
PDBModel.concat(chains_before,r1)

In [ ]:
m_chain.writePdb('capped_non_standard.pdb')
non_standard.writePdb('uncapped_non_standard.pdb')

In [ ]:
model = non_standard
chain = 0

import Biskit.tools as t
from Biskit import PDBModel
F_nme_cap = t.dataRoot() + '/amber/leap/nme_cap.pdb'
m_nme   = PDBModel( F_nme_cap )

chains_before = model.takeChains( range(chain), breaks=1 )
m_chain       = model.takeChains( [chain], breaks=1 )
chains_after  = model.takeChains( range(chain+1, model.lenChains(1)),
                                  breaks=1 )

m_term  = m_chain.resModels()[-1]

## rename overhanging residue in cap PDB, renumber cap residue
for a in m_nme:
    if a['residue_name'] != 'NME':
        a['residue_name'] = m_term.atoms['residue_name'][0]
    else:
        a['residue_number'] = m_term.atoms['residue_number'][0]+1
        a['chain_id']       = m_term.atoms['chain_id'][0]
        a['segment_id']     = m_term.atoms['segment_id'][0]

## chain should not have any terminal O after capping
m_chain.remove( ['OXT'] )            

## fit cap onto last residue of chain
m_nme = m_nme.magicFit( m_term )

## concat cap on chain
m_chain = m_chain.concat( m_nme.resModels()[-1] )

## should be obsolete now
if getattr( m_chain, '_PDBModel__terAtoms', []) != []:
    m_chain._PDBModel__terAtoms = [ len( m_chain ) - 1 ]

## re-assemble whole model
chains_before.concat( m_chain, chains_after )


In [ ]:
m=chains_before

In [ ]:
m_chain.residues['biomol']

In [ ]:
chains_before.concat(m_chain)

In [ ]:
non_standard.resMap()
non_standard.addChainId()
non_standard[0:5]
#parm_build =Biskit.AmberParmBuilder(gfp_raw)
#parm_build.capNME([r1,r2],1)
#c2=gfp_raw.takeChains([1])

In [ ]:
import Biskit.tools as t
t.dataRoot()

The procedure for preparing a qm/mm calculation is as follows:

Select the Chain we want to perform the calculation on (in general pick the first one)

Standard residues can be handled automatically by amber, gromacs et al. but non-standard residues need to be parameterised.

First extract the non-standard residues to a separate pdb file
Now construct a library file detailing, the connectivity, atom types and partial charges using this pdb file.

We need to determine correct placement of hydrogens and protonation states.

Finally construct a gaussian input file using these.

To select the chain we can use biopython or the openMM python api

To calculate standard connectivities we can use openMM, for non standard we could try using molmod (though may need to visually inspect the results)

To determine protonation states we can use propka more specifically [pdb2pqr](http://www.ics.uci.edu/~dock/pdb2pqr/userguide.html#using), see also the pyMol [propka](http://www.pymolwiki.org/index.php/Propka) script

Hydrogen placement can be determined in many different ways.

Parameterisation can use the tleap console (or the xleap gui if we are doing it manually).

Hydrogen placement can be done in many different ways.

In [ ]:
s=' C/1=N\CC/N=C/c(c2)cc(\C=N\CC/N=C/3)cc2\C=N\CC/N=C/c(c4)cc(\C=N\CC/N=C/5)cc4\C=N\CC/N=C/c(c6)cc1cc6\C=N\CC/N=C/c(c7)cc5cc37'

In [ ]:
t=s.replace('CC','C*CCCCC*')

In [ ]:
t

In [ ]:
'{blah}{blah2}'.format(blah=10,blah2='xyz')

first_count = 7
second_count = 7 
first_star = True
new_str = []

for c in t:
    if c=='*' and first_star:
        first_count=first_count+1
        first_star=False
        replacement_str = str(first_count)
        if first_count >= 10:
            replacement_str = '%' + replacement_str
        new_str.append(replacement_str)
    elif c=='*' and not first_star:
        second_count+=1
        first_star=True
        replacement_str2 = str(second_count)
        if second_count >= 10:
            replacement_str2 = '%' + replacement_str2
        new_str.append(replacement_str2)
    else:
        new_str.append(c)

''.join(new_str)